In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict
from sklearn.model_selection import KFold
import time

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the data

In [4]:
ml_ratings_df = pd.read_csv(os.path.join("data", "movielens_small", "ratings.csv")).rename(columns={'userId': 'user_id', 'movieId': 'item_id'})
ml_movies_df = pd.read_csv(os.path.join("data", "movielens_small", "movies.csv")).rename(columns={'movieId': 'item_id'})
ml_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')
ml_df.head(10)

display(HTML(ml_movies_df.head(10).to_html()))

# Filter the data to reduce the number of movies
seed = 6789
rng = np.random.RandomState(seed=seed)
left_ids = rng.choice(ml_movies_df['item_id'], size=1000, replace=False)

ml_ratings_df = ml_ratings_df.loc[ml_ratings_df['item_id'].isin(left_ids)]
ml_movies_df = ml_movies_df.loc[ml_movies_df['item_id'].isin(left_ids)]
ml_df = ml_df.loc[ml_df['item_id'].isin(left_ids)]

print("Number of chosen interactions: {}".format(len(ml_ratings_df)))

,item_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Number of chosen interactions: 9692


# Recommender class

Remark: Docstrings written in reStructuredText (reST) used by Sphinx to automatically generate code documentation. It is also used by default by PyCharm (type triple quotes after defining a class or a method and hit enter).

In [5]:
class Recommender(object):
    """
    Base recommender class.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        
        :param int seed: Seed for the random number generator.
        """
        pass
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        pass
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            user_recommendations = pd.DataFrame({'user_id': user['user_id'],
                                                 'item_id': [-1] * n_recommendations,
                                                 'score': [3.0] * n_recommendations})

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations

# Evaluation measures

## Explicit feedback - ratings

### MAE - Mean Absolute error

<center>
$$
    MAE = \frac{\sum_{i}^N |\hat{r}_i - r_i|}{N}
$$
</center>

where $\hat{r}_i$ are the predicted ratings and $r_i$ are the real ratings and $N$ is the number of items in the test set.

    + Easily interpretable.
    - Non-differentiable which makes it unsuitable for gradient descent.
    - The scale of errors dependent on data which reduces comparability between different datasets.

In [4]:
def mae(r_pred, r_real):
    return np.sum(np.abs(r_pred - r_real)) / len(r_pred)

# Test

# Very small differences
print("MAE = {:.3f}".format(mae(np.array([2.99, 1.98, 3.99, 4.97, 1.01]), np.array([3, 2, 4, 5, 1]))))
# Small differences
print("MAE = {:.3f}".format(mae(np.array([2.8, 1.7, 3.8, 4.6, 1.6]), np.array([3, 2, 4, 5, 1]))))
# Large differences
print("MAE = {:.3f}".format(mae(np.array([1.1, 4.2, 2.8, 3.2, 3.6]), np.array([3, 2, 4, 5, 1]))))
# Medium differences with one large difference
print("MAE = {:.3f}".format(mae(np.array([2.1, 1.2, 3.8, 4.2, 3.6]), np.array([3, 2, 4, 5, 1]))))
# Small differences with one very large difference
print("MAE = {:.3f}".format(mae(np.array([2.8, 1.7, 3.8, 4.6, 4.6]), np.array([3, 2, 4, 5, 1]))))

MAE = 0.016
MAE = 0.340
MAE = 1.940
MAE = 1.060
MAE = 0.940


### RMSE - Root Mean Squared Error

<center>
$$
    RMSE = \sqrt{\frac{\sum_{i}^N (\hat{r}_i - r_i)^2}{N}}
$$
</center>

where $\hat{r}_i$ are the predicted ratings and $r_i$ are the real ratings and $N$ is the number of items in the test set.

    + Very well-behaved analytically and therefore extensively used to train models, especially neural networks.
    - The scale of errors dependent on data which reduced comparability between different datasets.

In [5]:
def rmse(r_pred, r_real):
    return np.sqrt(np.sum(np.power(r_pred - r_real, 2)) / len(r_pred))

# Test

# Very small differences
print("RMSE = {:.3f}".format(rmse(np.array([2.99, 1.98, 3.99, 4.97, 1.01]), np.array([3, 2, 4, 5, 1]))))
# Small differences
print("RMSE = {:.3f}".format(rmse(np.array([2.8, 1.7, 3.8, 4.6, 1.6]), np.array([3, 2, 4, 5, 1]))))
# Large differences
print("RMSE = {:.3f}".format(rmse(np.array([1.1, 4.2, 2.8, 3.2, 3.6]), np.array([3, 2, 4, 5, 1]))))
# Medium differences with one large difference
print("RMSE = {:.3f}".format(rmse(np.array([2.1, 1.2, 3.8, 4.2, 3.6]), np.array([3, 2, 4, 5, 1]))))
# Small differences with one very large difference
print("RMSE = {:.3f}".format(rmse(np.array([2.8, 1.7, 3.8, 4.6, 4.6]), np.array([3, 2, 4, 5, 1]))))

RMSE = 0.018
RMSE = 0.371
RMSE = 1.994
RMSE = 1.333
RMSE = 1.630


### MRE - Mean Relative Error (more often called MAPE)

<center>
$$
    MRE = \frac{1}{N} \sum_{i}^N \frac{|\hat{r}_i - r_i|}{|r_i|}
$$
</center>

where $\hat{r}_i$ are the predicted ratings and $r_i$ are the real ratings and $N$ is the number of items in the test set.

    + Easily interpretable (average percentage error) and with a meaning understandable for business.
    - Blows up when there are values close to zero among the predicted values.

In [6]:
def mre(r_pred, r_real):
    return 1 / len(r_pred) * np.sum(np.abs(r_pred - r_real) / np.abs(r_real))

# Test

# Very small differences
print("MRE = {:.3f}".format(mre(np.array([2.99, 1.98, 3.99, 4.97, 1.01]), np.array([3, 2, 4, 5, 1]))))
# Small differences
print("MRE = {:.3f}".format(mre(np.array([2.8, 1.7, 3.8, 4.6, 1.6]), np.array([3, 2, 4, 5, 1]))))
# Large differences
print("MRE = {:.3f}".format(mre(np.array([1.1, 4.2, 2.8, 3.2, 3.6]), np.array([3, 2, 4, 5, 1]))))
# Medium differences with one large difference
print("MRE = {:.3f}".format(mre(np.array([2.1, 1.2, 3.8, 4.2, 3.6]), np.array([3, 2, 4, 5, 1]))))
# Small differences with one very large difference for a small base value
print("MRE = {:.3f}".format(mre(np.array([2.8, 1.7, 3.8, 4.6, 4.6]), np.array([3, 2, 4, 5, 1]))))

MRE = 0.006
MRE = 0.189
MRE = 0.999
MRE = 0.702
MRE = 0.789


### TRE - Total Relative Error

<center>
$$
    TRE = \frac{\sum_{i}^N |\hat{r}_i - r_i|}{\sum_{i}^N |r_i|}
$$
</center>

where $\hat{r}_i$ are the predicted ratings and $r_i$ are the real ratings and $N$ is the number of items in the test set.

    + Easily interpretable (total percentage error) and with a meaning understandable for business.
    + Reliable even for very small predicted values.
    - Does not distinguish between a case when one prediction is very bad and other are very good and a case when all predictions are mediocre.

In [7]:
def tre(r_pred, r_real):
    return np.sum(np.abs(r_pred - r_real)) / np.sum(np.abs(r_real))

# Test

# Very small differences
print("TRE = {:.3f}".format(tre(np.array([2.99, 1.98, 3.99, 4.97, 1.01]), np.array([3, 2, 4, 5, 1]))))
# Small differences
print("TRE = {:.3f}".format(tre(np.array([2.8, 1.7, 3.8, 4.6, 1.6]), np.array([3, 2, 4, 5, 1]))))
# Large differences
print("TRE = {:.3f}".format(tre(np.array([1.1, 4.2, 2.8, 3.2, 3.6]), np.array([3, 2, 4, 5, 1]))))
# Medium differences with one large difference
print("TRE = {:.3f}".format(tre(np.array([2.1, 1.2, 3.8, 4.2, 3.6]), np.array([3, 2, 4, 5, 1]))))
# Small differences with one very large difference for a small base value
print("TRE = {:.3f}".format(tre(np.array([2.8, 1.7, 3.8, 4.6, 4.6]), np.array([3, 2, 4, 5, 1]))))

TRE = 0.005
TRE = 0.113
TRE = 0.647
TRE = 0.353
TRE = 0.313


## Implicit feedback - binary indicators of interactions

### HR@n - Hit Ratio 
How many hits did we score in the first n recommendations.
<br/>
<br/>
<center>
$$
    \text{HR@}n = \frac{\sum_{u} \sum_{i \in I_u} r_{u, i} \cdot 1_{\hat{D}_n(u)}(i)}{M}
$$
</center>

where:
  * $r_{u, i}$ is $1$ if there was an interaction between user $u$ and item $i$ in the test set and $0$ otherwise, 
  * $\hat{D}_n$ is the set of the first $n$ recommendations for user $u$, 
  * $1_{\hat{D}_n}(i)$ is $1$ if and only if $i \in \hat{D}_n$, otherwise it's equal to $0$,
  * $M$ is the number of users.


    + Easily interpretable.
    - Does not take the rank of each recommendation into account.

In [8]:
def hr(recommendations, real_interactions, n=1):
    """
    Assumes recommendations are ordered by user_id and then by score.
    """
    # Transform real_interactions to a dict for a large speed-up
    rui = defaultdict(lambda: 0)
    
    for idx, row in real_interactions.iterrows():
        rui[(row['user_id'], row['item_id'])] = 1
        
    hr = 0.0
    
    previous_user_id = -1
    rank = 0
    for idx, row in recommendations.iterrows():
        if previous_user_id == row['user_id']:
            rank += 1
        else:
            rank = 1
            
        if rank <= n:
            hr += rui[(row['user_id'], row['item_id'])]
        
        previous_user_id = row['user_id']
    
    hr /= len(recommendations['user_id'].unique())
    
    return hr

In [9]:
# Case 1
    
real_interactions = pd.DataFrame(
    [
        [1, 45],
        [1, 22],
        [1, 77],
        [2, 13],
        [2, 77]
        
    ], columns=['user_id', 'item_id'])

display(real_interactions)
    
recommendations = pd.DataFrame(
    [
        [1, 45, 0.9],
        [1, 13, 0.8],
        [1, 22, 0.71],
        [1, 77, 0.55],
        [1, 9, 0.52],
        [2, 11, 0.85],
        [2, 13, 0.69],
        [2, 25, 0.64],
        [2, 6, 0.60],
        [2, 77, 0.53]
        
    ], columns=['user_id', 'item_id', 'score'])

display(recommendations)
    
print("HR@3 = {:.4f}".format(hr(recommendations, real_interactions, n=3)))

,user_id,item_id
0,1,45
1,1,22
2,1,77
3,2,13
4,2,77


,user_id,item_id,score
0,1,45,0.90
1,1,13,0.80
2,1,22,0.71
3,1,77,0.55
4,1,9,0.52
5,2,11,0.85
6,2,13,0.69
7,2,25,0.64
8,2,6,0.60
9,2,77,0.53


HR@3 = 1.5000


In [10]:
# Case 2
    
real_interactions = pd.DataFrame(
    [
        [1, 45],
        [1, 22],
        [1, 77],
        [2, 13],
        [2, 77]
        
    ], columns=['user_id', 'item_id'])

display(real_interactions)
    
recommendations = pd.DataFrame(
    [
        [1, 13, 0.9],
        [1, 45, 0.8],
        [1, 22, 0.71],
        [1, 77, 0.55],
        [1, 9, 0.52],
        [2, 11, 0.85],
        [2, 13, 0.69],
        [2, 25, 0.64],
        [2, 6, 0.60],
        [2, 77, 0.53]
        
    ], columns=['user_id', 'item_id', 'score'])

display(recommendations)
    
print("HR@3 = {:.4f}".format(hr(recommendations, real_interactions, n=3)))

,user_id,item_id
0,1,45
1,1,22
2,1,77
3,2,13
4,2,77


,user_id,item_id,score
0,1,13,0.90
1,1,45,0.80
2,1,22,0.71
3,1,77,0.55
4,1,9,0.52
5,2,11,0.85
6,2,13,0.69
7,2,25,0.64
8,2,6,0.60
9,2,77,0.53


HR@3 = 1.5000


### NDCG@n - Normalized Discounted Cumulative Gain

How many hits did we score in the first n recommendations discounted by the position of each recommendation.
<br/>
<br/>
<center>
$$
    \text{NDCG@}n = \frac{\sum_{u} \sum_{i \in I_u} \frac{r_{u, i}}{log\left(1 + v_{\hat{D}_n(u)}(i)\right)}}{M}
$$
</center>

where:
  * $r_{u, i}$ is $1$ if there was an interaction between user $u$ and item $i$ in the test set and $0$ otherwise, 
  * $\hat{D}_n(u)$ is the set of the first $n$ recommendations for user $u$, 
  * $v_{\hat{D}_n(u)}(i)$ is the position of item $i$ in recommendations $\hat{D}_n$,
  * $M$ is the number of users.


    - Takes the rank of each recommendation into account.

In [8]:
def ndcg(recommendations, real_interactions, n=1):
    """
    Assumes recommendations are ordered by user_id and then by score.
    """
    # Transform real_interactions to a dict for a large speed-up
    rui = defaultdict(lambda: 0)
    
    for idx, row in real_interactions.iterrows():
        rui[(row['user_id'], row['item_id'])] = 1
        
    ndcg = 0.0
    
    previous_user_id = -1
    rank = 0
    for idx, row in recommendations.iterrows():
        if previous_user_id == row['user_id']:
            rank += 1
        else:
            rank = 1
            
        if rank <= n:
            ndcg += rui[(row['user_id'], row['item_id'])] / np.log2(1 + rank)
        print(ndcg)
        
        previous_user_id = row['user_id']
    
    ndcg /= len(recommendations['user_id'].unique())
    
    return ndcg

In [9]:
# Case 1
    
real_interactions = pd.DataFrame(
    [
        [1, 45],
        [1, 22],
        [1, 77],
        [2, 13],
        [2, 77]
        
    ], columns=['user_id', 'item_id'])

display(real_interactions)
    
recommendations = pd.DataFrame(
    [
        [1, 45, 0.9],
        [1, 13, 0.8],
        [1, 22, 0.71],
        [1, 77, 0.55],
        [1, 9, 0.52],
        [2, 11, 0.85],
        [2, 13, 0.69],
        [2, 25, 0.64],
        [2, 6, 0.60],
        [2, 77, 0.53]
        
    ], columns=['user_id', 'item_id', 'score'])

display(recommendations)
    
print("NDCG@3 = {:.4f}".format(ndcg(recommendations, real_interactions, n=3)))

,user_id,item_id
0,1,45
1,1,22
2,1,77
3,2,13
4,2,77


,user_id,item_id,score
0,1,45,0.90
1,1,13,0.80
2,1,22,0.71
3,1,77,0.55
4,1,9,0.52
5,2,11,0.85
6,2,13,0.69
7,2,25,0.64
8,2,6,0.60
9,2,77,0.53


1.0
1.0
1.5
1.5
1.5
1.5
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
NDCG@3 = 1.0655


In [13]:
# Case 2
    
real_interactions = pd.DataFrame(
    [
        [1, 45],
        [1, 22],
        [1, 77],
        [2, 13],
        [2, 77]
        
    ], columns=['user_id', 'item_id'])

display(real_interactions)
    
recommendations = pd.DataFrame(
    [
        [1, 13, 0.9],
        [1, 45, 0.8],
        [1, 22, 0.71],
        [1, 77, 0.55],
        [1, 9, 0.52],
        [2, 11, 0.85],
        [2, 13, 0.69],
        [2, 25, 0.64],
        [2, 6, 0.60],
        [2, 77, 0.53]
        
    ], columns=['user_id', 'item_id', 'score'])

display(recommendations)
    
print("NDCG@3 = {:.4f}".format(ndcg(recommendations, real_interactions, n=3)))

,user_id,item_id
0,1,45
1,1,22
2,1,77
3,2,13
4,2,77


,user_id,item_id,score
0,1,13,0.90
1,1,45,0.80
2,1,22,0.71
3,1,77,0.55
4,1,9,0.52
5,2,11,0.85
6,2,13,0.69
7,2,25,0.64
8,2,6,0.60
9,2,77,0.53


NDCG@3 = 0.8809


# Testing routines (offline)

## Train and test set split

### Explicit feedback

In [15]:
def evaluate_train_test_split_explicit(recommender, interactions_df, items_df, seed=6789):
    rng = np.random.RandomState(seed=seed)
    
    if type(interactions_df) == dict:
        # If interactions_df is a dict with already split data, use the split
        interactions_df_train = interactions_df['train']
        interactions_df_test = interactions_df['test']
    else:    
        # Otherwise split the dataset into train and test

        shuffle = np.arange(len(interactions_df))
        rng.shuffle(shuffle)
        shuffle = list(shuffle)

        train_test_split = 0.8
        split_index = int(len(interactions_df) * train_test_split)

        interactions_df_train = interactions_df.iloc[shuffle[:split_index]]
        interactions_df_test = interactions_df.iloc[shuffle[split_index:]]
    
    # Train the recommender
    
    recommender.fit(interactions_df_train, None, items_df)
    
#     print(recommender)
#     print(interactions_df)
#     print(items_df)
    
    # Gather predictions
    
    r_pred = []
    
    for idx, row in interactions_df_test.iterrows():
        users_df = pd.DataFrame([row['user_id']], columns=['user_id'])
        eval_items_df = pd.DataFrame([row['item_id']], columns=['item_id'])
        eval_items_df = pd.merge(eval_items_df, items_df, on='item_id')
        recommendations = recommender.recommend(users_df, eval_items_df, n_recommendations=1)
        
        r_pred.append(recommendations.iloc[0]['score'])
    
    # Gather real ratings
    
    r_real = np.array(interactions_df_test['rating'].tolist())
    
    # Return evaluation metrics
    
    return rmse(r_pred, r_real), mre(r_pred, r_real), tre(r_pred, r_real)

recommender = Recommender()

results = [['BaseRecommender'] + list(evaluate_train_test_split_explicit(
    recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(HTML(results.to_html()))

      item_id                               title  \
3           4            Waiting to Exhale (1995)   
11         12  Dracula: Dead and Loving It (1995)   
46         50          Usual Suspects, The (1995)   
66         74                 Bed of Roses (1996)   
70         78          Crossing Guard, The (1995)   
...       ...                                 ...   
9697   184987            A Wrinkle in Time (2018)   
9704   185473                     Blockers (2018)   
9721   189381                     SuperFly (2018)   
9725   190207                         Tilt (2011)   
9727   190213                    John From (2015)   

                                 genres  
3                  Comedy|Drama|Romance  
11                        Comedy|Horror  
46               Crime|Mystery|Thriller  
66                        Drama|Romance  
70          Action|Crime|Drama|Thriller  
...                                 ...  
9697  Adventure|Children|Fantasy|Sci-Fi  
9704                       

   item_id                     title                 genres
0   7438.0  Kill Bill: Vol. 2 (2004)  Action|Drama|Thriller
      item_id                               title  \
3           4            Waiting to Exhale (1995)   
11         12  Dracula: Dead and Loving It (1995)   
46         50          Usual Suspects, The (1995)   
66         74                 Bed of Roses (1996)   
70         78          Crossing Guard, The (1995)   
...       ...                                 ...   
9697   184987            A Wrinkle in Time (2018)   
9704   185473                     Blockers (2018)   
9721   189381                     SuperFly (2018)   
9725   190207                         Tilt (2011)   
9727   190213                    John From (2015)   

                                 genres  
3                  Comedy|Drama|Romance  
11                        Comedy|Horror  
46               Crime|Mystery|Thriller  
66                        Drama|Romance  
70          Action|Crime|Drama|Th

   item_id                title                     genres
0     95.0  Broken Arrow (1996)  Action|Adventure|Thriller
      item_id                               title  \
3           4            Waiting to Exhale (1995)   
11         12  Dracula: Dead and Loving It (1995)   
46         50          Usual Suspects, The (1995)   
66         74                 Bed of Roses (1996)   
70         78          Crossing Guard, The (1995)   
...       ...                                 ...   
9697   184987            A Wrinkle in Time (2018)   
9704   185473                     Blockers (2018)   
9721   189381                     SuperFly (2018)   
9725   190207                         Tilt (2011)   
9727   190213                    John From (2015)   

                                 genres  
3                  Comedy|Drama|Romance  
11                        Comedy|Horror  
46               Crime|Mystery|Thriller  
66                        Drama|Romance  
70          Action|Crime|Drama|Thri

    item_id                 title            genres
0  158872.0  Sausage Party (2016)  Animation|Comedy
      item_id                               title  \
3           4            Waiting to Exhale (1995)   
11         12  Dracula: Dead and Loving It (1995)   
46         50          Usual Suspects, The (1995)   
66         74                 Bed of Roses (1996)   
70         78          Crossing Guard, The (1995)   
...       ...                                 ...   
9697   184987            A Wrinkle in Time (2018)   
9704   185473                     Blockers (2018)   
9721   189381                     SuperFly (2018)   
9725   190207                         Tilt (2011)   
9727   190213                    John From (2015)   

                                 genres  
3                  Comedy|Drama|Romance  
11                        Comedy|Horror  
46               Crime|Mystery|Thriller  
66                        Drama|Romance  
70          Action|Crime|Drama|Thriller  
...    

   item_id                                    title                   genres
0   6713.0  Millennium Actress (Sennen joyû) (2001)  Animation|Drama|Romance
      item_id                               title  \
3           4            Waiting to Exhale (1995)   
11         12  Dracula: Dead and Loving It (1995)   
46         50          Usual Suspects, The (1995)   
66         74                 Bed of Roses (1996)   
70         78          Crossing Guard, The (1995)   
...       ...                                 ...   
9697   184987            A Wrinkle in Time (2018)   
9704   185473                     Blockers (2018)   
9721   189381                     SuperFly (2018)   
9725   190207                         Tilt (2011)   
9727   190213                    John From (2015)   

                                 genres  
3                  Comedy|Drama|Romance  
11                        Comedy|Horror  
46               Crime|Mystery|Thriller  
66                        Drama|Romance  

...

   item_id                   title                                  genres
0  48774.0  Children of Men (2006)  Action|Adventure|Drama|Sci-Fi|Thriller
      item_id                               title  \
3           4            Waiting to Exhale (1995)   
11         12  Dracula: Dead and Loving It (1995)   
46         50          Usual Suspects, The (1995)   
66         74                 Bed of Roses (1996)   
70         78          Crossing Guard, The (1995)   
...       ...                                 ...   
9697   184987            A Wrinkle in Time (2018)   
9704   185473                     Blockers (2018)   
9721   189381                     SuperFly (2018)   
9725   190207                         Tilt (2011)   
9727   190213                    John From (2015)   

                                 genres  
3                  Comedy|Drama|Romance  
11                        Comedy|Horror  
46               Crime|Mystery|Thriller  
66                        Drama|Romance  
70 

   item_id            title                              genres
0    145.0  Bad Boys (1995)  Action|Comedy|Crime|Drama|Thriller
      item_id                               title  \
3           4            Waiting to Exhale (1995)   
11         12  Dracula: Dead and Loving It (1995)   
46         50          Usual Suspects, The (1995)   
66         74                 Bed of Roses (1996)   
70         78          Crossing Guard, The (1995)   
...       ...                                 ...   
9697   184987            A Wrinkle in Time (2018)   
9704   185473                     Blockers (2018)   
9721   189381                     SuperFly (2018)   
9725   190207                         Tilt (2011)   
9727   190213                    John From (2015)   

                                 genres  
3                  Comedy|Drama|Romance  
11                        Comedy|Horror  
46               Crime|Mystery|Thriller  
66                        Drama|Romance  
70          Action|Crime|

   item_id                        title                  genres
0  46578.0  Little Miss Sunshine (2006)  Adventure|Comedy|Drama
      item_id                               title  \
3           4            Waiting to Exhale (1995)   
11         12  Dracula: Dead and Loving It (1995)   
46         50          Usual Suspects, The (1995)   
66         74                 Bed of Roses (1996)   
70         78          Crossing Guard, The (1995)   
...       ...                                 ...   
9697   184987            A Wrinkle in Time (2018)   
9704   185473                     Blockers (2018)   
9721   189381                     SuperFly (2018)   
9725   190207                         Tilt (2011)   
9727   190213                    John From (2015)   

                                 genres  
3                  Comedy|Drama|Romance  
11                        Comedy|Horror  
46               Crime|Mystery|Thriller  
66                        Drama|Romance  
70          Action|Crime|

    item_id                          title                     genres
0  146684.0  Cosmic Scrat-tastrophe (2015)  Animation|Children|Comedy
      item_id                               title  \
3           4            Waiting to Exhale (1995)   
11         12  Dracula: Dead and Loving It (1995)   
46         50          Usual Suspects, The (1995)   
66         74                 Bed of Roses (1996)   
70         78          Crossing Guard, The (1995)   
...       ...                                 ...   
9697   184987            A Wrinkle in Time (2018)   
9704   185473                     Blockers (2018)   
9721   189381                     SuperFly (2018)   
9725   190207                         Tilt (2011)   
9727   190213                    John From (2015)   

                                 genres  
3                  Comedy|Drama|Romance  
11                        Comedy|Horror  
46               Crime|Mystery|Thriller  
66                        Drama|Romance  
70          A

NameError: name 'rmse' is not defined

### Implicit feedback

In [15]:
def evaluate_train_test_split_implicit(recommender, interactions_df, items_df, seed=6789):
    rng = np.random.RandomState(seed=seed)
    
    if type(interactions_df) == 'dict':
        # If interactions_df is a dict with already split data, use the split
        interactions_df_train = interactions_df['train']
        interactions_df_test = interactions_df['test']
    else:    
        # Otherwise split the dataset into train and test

        shuffle = np.arange(len(interactions_df))
        rng.shuffle(shuffle)
        shuffle = list(shuffle)

        train_test_split = 0.8
        split_index = int(len(interactions_df) * train_test_split)

        interactions_df_train = interactions_df.iloc[shuffle[:split_index]]
        interactions_df_test = interactions_df.iloc[shuffle[split_index:]]
    
    hr_1 = []
    hr_3 = []
    hr_5 = []
    hr_10 = []
    ndcg_1 = []
    ndcg_3 = []
    ndcg_5 = []
    ndcg_10 = []
    
    # Train the recommender
    
    recommender.fit(interactions_df_train, None, items_df)
    
    # Make recommendations for each user in the test set and calculate the metric 
    # against all items of that user in the test set
    
    test_user_interactions = interactions_df_test.groupby(by='user_id')
    
    for user_id, user_interactions in test_user_interactions:
        
        recommendations = recommender.recommend(pd.DataFrame([user_id], columns=['user_id']), 
                                                items_df, n_recommendations=10)
        
        hr_1.append(hr(recommendations, user_interactions, n=1))
        hr_3.append(hr(recommendations, user_interactions, n=3))
        hr_5.append(hr(recommendations, user_interactions, n=5))
        hr_10.append(hr(recommendations, user_interactions, n=10))
        ndcg_1.append(ndcg(recommendations, user_interactions, n=1))
        ndcg_3.append(ndcg(recommendations, user_interactions, n=3))
        ndcg_5.append(ndcg(recommendations, user_interactions, n=5))
        ndcg_10.append(ndcg(recommendations, user_interactions, n=10))
                
    hr_1 = np.mean(hr_1)
    hr_3 = np.mean(hr_3)
    hr_5 = np.mean(hr_5)
    hr_10 = np.mean(hr_10)
    ndcg_1 = np.mean(ndcg_1)
    ndcg_3 = np.mean(ndcg_3)
    ndcg_5 = np.mean(ndcg_5)
    ndcg_10 = np.mean(ndcg_10)
    
    return hr_1, hr_3, hr_5, hr_10, ndcg_1, ndcg_3, ndcg_5, ndcg_10

recommender = Recommender()

results = [['BaseRecommender'] + list(evaluate_train_test_split_implicit(
    recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,BaseRecommender,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Leave-one-out, leave-k-out, cross-validation

### Explicit feedback

In [16]:
def evaluate_leave_one_out_explicit(recommender, interactions_df, items_df, max_evals=300, seed=6789):
    rng = np.random.RandomState(seed=seed)
    
    # Prepare splits of the datasets
    kf = KFold(n_splits=len(interactions_df), random_state=rng, shuffle=True)
    
    # For each split of the dataset train the recommender, generate recommendations and evaluate
    
    r_pred = []
    r_real = []
    n_eval = 1
    for train_index, test_index in kf.split(interactions_df.index):
        interactions_df_train = interactions_df.loc[interactions_df.index[train_index]]
        interactions_df_test = interactions_df.loc[interactions_df.index[test_index]]

        recommender.fit(interactions_df_train, None, items_df)
        recommendations = recommender.recommend(interactions_df_test.loc[:, ['user_id']], 
                                                items_df.loc[items_df['item_id'] == interactions_df_test.iloc[0]['item_id']])
        
        r_pred.append(recommendations.iloc[0]['score'])
        r_real.append(interactions_df_test.iloc[0]['rating'])
        
        if n_eval == max_evals:
            break
        n_eval += 1
        
    r_pred = np.array(r_pred)
    r_real = np.array(r_real)
        
    # Return evaluation metrics
    
    return rmse(r_pred, r_real), mre(r_pred, r_real), tre(r_pred, r_real)

recommender = Recommender()

results = [['BaseRecommender'] + list(evaluate_leave_one_out_explicit(
    recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(HTML(results.to_html()))

,Recommender,RMSE,MRE,TRE
0,BaseRecommender,1.161536,0.382929,0.267866


### Implicit feedback

In [17]:
def evaluate_leave_one_out_implicit(recommender, interactions_df, items_df, max_evals=300, seed=6789):
    rng = np.random.RandomState(seed=seed)
    
    # Prepare splits of the datasets
    kf = KFold(n_splits=len(interactions_df), random_state=rng, shuffle=True)
    
    hr_1 = []
    hr_3 = []
    hr_5 = []
    hr_10 = []
    ndcg_1 = []
    ndcg_3 = []
    ndcg_5 = []
    ndcg_10 = []
    
    # For each split of the dataset train the recommender, generate recommendations and evaluate
    
    n_eval = 1
    for train_index, test_index in kf.split(interactions_df.index):
        interactions_df_train = interactions_df.loc[interactions_df.index[train_index]]
        interactions_df_test = interactions_df.loc[interactions_df.index[test_index]]
                
        recommender.fit(interactions_df_train, None, items_df)
        recommendations = recommender.recommend(interactions_df_test.loc[:, ['user_id']], items_df, n_recommendations=10)
        
        hr_1.append(hr(recommendations, interactions_df_test, n=1))
        hr_3.append(hr(recommendations, interactions_df_test, n=3))
        hr_5.append(hr(recommendations, interactions_df_test, n=5))
        hr_10.append(hr(recommendations, interactions_df_test, n=10))
        ndcg_1.append(ndcg(recommendations, interactions_df_test, n=1))
        ndcg_3.append(ndcg(recommendations, interactions_df_test, n=3))
        ndcg_5.append(ndcg(recommendations, interactions_df_test, n=5))
        ndcg_10.append(ndcg(recommendations, interactions_df_test, n=10))
        
        if n_eval == max_evals:
            break
        n_eval += 1
        
    hr_1 = np.mean(hr_1)
    hr_3 = np.mean(hr_3)
    hr_5 = np.mean(hr_5)
    hr_10 = np.mean(hr_10)
    ndcg_1 = np.mean(ndcg_1)
    ndcg_3 = np.mean(ndcg_3)
    ndcg_5 = np.mean(ndcg_5)
    ndcg_10 = np.mean(ndcg_10)
    
    return hr_1, hr_3, hr_5, hr_10, ndcg_1, ndcg_3, ndcg_5, ndcg_10

recommender = Recommender()

results = [['BaseRecommender'] + list(evaluate_leave_one_out_implicit(
    recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,BaseRecommender,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Examples of evaluation

## Explicit feedback

### Train-test split test

In [18]:
from recommenders.basic_recommenders import MostPopularRecommender
from recommenders.basic_recommenders import HighestRatedRecommender
from recommenders.basic_content_based_recommenders import LinearRegressionRecommender
from recommenders.basic_content_based_recommenders import SVRRecommender
from recommenders.nearest_neighbors_recommender import ItemBasedCosineNearestNeighborsRecommender

highest_rated_recommender = HighestRatedRecommender()
lr_recommender = LinearRegressionRecommender()
svr_recommender = SVRRecommender()

recommenders = [highest_rated_recommender, lr_recommender, svr_recommender]

all_results = []

t0 = time.time()

for recommender in recommenders:
    results = [[type(recommender).__name__] + list(evaluate_train_test_split_explicit(
        recommender, ml_ratings_df, ml_movies_df))]

    results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])
    all_results.append(results)

    display(results)
    
all_results = pd.concat(all_results).reset_index(drop=True)
display(all_results)

print('Total evaluation time: {}'.format(time.time() - t0))

,Recommender,RMSE,MRE,TRE
0,HighestRatedRecommender,1.272994,0.481631,0.277884


,Recommender,RMSE,MRE,TRE
0,LinearRegressionRecommender,1.016594,0.348461,0.230529


,Recommender,RMSE,MRE,TRE
0,SVRRecommender,0.998426,0.344447,0.2216


,Recommender,RMSE,MRE,TRE
0,HighestRatedRecommender,1.272994,0.481631,0.277884
1,LinearRegressionRecommender,1.016594,0.348461,0.230529
2,SVRRecommender,0.998426,0.344447,0.221600


Total evaluation time: 130.65900230407715


### Leave-one-out test

In [19]:
from recommenders.basic_recommenders import MostPopularRecommender
from recommenders.basic_recommenders import HighestRatedRecommender
from recommenders.basic_content_based_recommenders import LinearRegressionRecommender
from recommenders.basic_content_based_recommenders import SVRRecommender
from recommenders.nearest_neighbors_recommender import ItemBasedCosineNearestNeighborsRecommender

highest_rated_recommender = HighestRatedRecommender()
lr_recommender = LinearRegressionRecommender()
svr_recommender = SVRRecommender()

recommenders = [highest_rated_recommender, lr_recommender, svr_recommender]

all_results = []

t0 = time.time()

for recommender in recommenders:
    results = [[type(recommender).__name__] + list(evaluate_leave_one_out_explicit(
        recommender, ml_ratings_df, ml_movies_df))]

    results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])
    all_results.append(results)

    display(results)
    
all_results = pd.concat(all_results).reset_index(drop=True)
display(all_results)

print('Total evaluation time: {}'.format(time.time() - t0))

,Recommender,RMSE,MRE,TRE
0,HighestRatedRecommender,1.380808,0.579605,0.313605


,Recommender,RMSE,MRE,TRE
0,LinearRegressionRecommender,1.045415,0.401399,0.242301


,Recommender,RMSE,MRE,TRE
0,SVRRecommender,1.030074,0.399686,0.235447


,Recommender,RMSE,MRE,TRE
0,HighestRatedRecommender,1.380808,0.579605,0.313605
1,LinearRegressionRecommender,1.045415,0.401399,0.242301
2,SVRRecommender,1.030074,0.399686,0.235447


Total evaluation time: 1840.7159786224365


## Implicit feedback

### Train-test split test

In [20]:
from recommenders.basic_recommenders import MostPopularRecommender
from recommenders.basic_recommenders import HighestRatedRecommender
from recommenders.basic_content_based_recommenders import LinearRegressionRecommender
from recommenders.basic_content_based_recommenders import SVRRecommender
from recommenders.nearest_neighbors_recommender import ItemBasedCosineNearestNeighborsRecommender

most_popular_recommender = MostPopularRecommender()
highest_rated_recommender = HighestRatedRecommender()
lr_recommender = LinearRegressionRecommender()
svr_recommender = SVRRecommender()
ibcnn_recommender = ItemBasedCosineNearestNeighborsRecommender(n_neighbors=30)

recommenders = [most_popular_recommender, highest_rated_recommender, lr_recommender, svr_recommender, ibcnn_recommender]

all_results = []

t0 = time.time()

for recommender in recommenders:
    results = [[type(recommender).__name__] + list(evaluate_train_test_split_implicit(
        recommender, ml_ratings_df, ml_movies_df))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])
    all_results.append(results)

    display(results)
    
all_results = pd.concat(all_results).reset_index(drop=True)
display(all_results)

print('Total evaluation time: {}'.format(time.time() - t0))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,MostPopularRecommender,0.076063,0.284116,0.440716,0.680089,0.076063,0.192392,0.256306,0.332798


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,HighestRatedRecommender,0.114094,0.239374,0.331096,0.568233,0.114094,0.186693,0.224823,0.302199


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionRecommender,0.0,0.006711,0.01566,0.080537,0.0,0.003942,0.007501,0.028281


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,SVRRecommender,0.006711,0.011186,0.013423,0.040268,0.006711,0.009241,0.010107,0.018459


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,ItemBasedCosineNearestNeighborsRecommender,0.14094,0.404922,0.610738,0.955257,0.14094,0.291677,0.376199,0.488492


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,MostPopularRecommender,0.076063,0.284116,0.440716,0.680089,0.076063,0.192392,0.256306,0.332798
1,HighestRatedRecommender,0.114094,0.239374,0.331096,0.568233,0.114094,0.186693,0.224823,0.302199
2,LinearRegressionRecommender,0.000000,0.006711,0.015660,0.080537,0.000000,0.003942,0.007501,0.028281
3,SVRRecommender,0.006711,0.011186,0.013423,0.040268,0.006711,0.009241,0.010107,0.018459
4,ItemBasedCosineNearestNeighborsRecommender,0.140940,0.404922,0.610738,0.955257,0.140940,0.291677,0.376199,0.488492


Total evaluation time: 569.9556179046631


### Leave-one-out test

In [21]:
from recommenders.basic_recommenders import MostPopularRecommender
from recommenders.basic_recommenders import HighestRatedRecommender
from recommenders.basic_content_based_recommenders import LinearRegressionRecommender
from recommenders.basic_content_based_recommenders import SVRRecommender
from recommenders.nearest_neighbors_recommender import ItemBasedCosineNearestNeighborsRecommender

most_popular_recommender = MostPopularRecommender()
highest_rated_recommender = HighestRatedRecommender()
lr_recommender = LinearRegressionRecommender()
svr_recommender = SVRRecommender()
ibcnn_recommender = ItemBasedCosineNearestNeighborsRecommender(n_neighbors=30)

recommenders = [most_popular_recommender, highest_rated_recommender, lr_recommender, svr_recommender, ibcnn_recommender]

all_results = []

t0 = time.time()

for recommender in recommenders:
    results = [[type(recommender).__name__] + list(evaluate_leave_one_out_implicit(
        recommender, ml_ratings_df, ml_movies_df))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])
    all_results.append(results)

    display(results)
    
all_results = pd.concat(all_results).reset_index(drop=True)
display(all_results)

print('Total evaluation time: {}'.format(time.time() - t0))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,MostPopularRecommender,0.026667,0.05,0.083333,0.163333,0.026667,0.040079,0.053997,0.079749


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,HighestRatedRecommender,0.013333,0.04,0.073333,0.113333,0.013333,0.02754,0.041603,0.054395


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionRecommender,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.003154


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,SVRRecommender,0.0,0.0,0.003333,0.013333,0.0,0.0,0.00129,0.004515


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,ItemBasedCosineNearestNeighborsRecommender,0.076667,0.17,0.233333,0.333333,0.076667,0.128134,0.154096,0.186049


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,MostPopularRecommender,0.026667,0.05,0.083333,0.163333,0.026667,0.040079,0.053997,0.079749
1,HighestRatedRecommender,0.013333,0.04,0.073333,0.113333,0.013333,0.027540,0.041603,0.054395
2,LinearRegressionRecommender,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.003154
3,SVRRecommender,0.000000,0.00,0.003333,0.013333,0.000000,0.000000,0.001290,0.004515
4,ItemBasedCosineNearestNeighborsRecommender,0.076667,0.17,0.233333,0.333333,0.076667,0.128134,0.154096,0.186049


Total evaluation time: 2517.236170053482
